In [ ]:
!pip install -U imbalanced-learn

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

root_path = 'gdrive/My Drive/Colab Notebooks/Deep Learning Labs/Framingham/'

print("Path root:", root_path)

Mounted at /content/gdrive
Path root: gdrive/My Drive/Colab Notebooks/Deep Learning Labs/Framingham/


In [38]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1.0,39.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,195.0,106.0,70.0,26.97,80.0,77.0,0.0
1,0.0,46.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,121.0,81.0,28.73,95.0,76.0,0.0
2,1.0,48.0,1.0,1.0,20.0,0.0,0.0,0.0,0.0,245.0,127.5,80.0,25.34,75.0,70.0,0.0
3,0.0,61.0,3.0,1.0,30.0,0.0,0.0,1.0,0.0,225.0,150.0,95.0,28.58,65.0,103.0,1.0
4,0.0,46.0,3.0,1.0,23.0,0.0,0.0,0.0,0.0,285.0,130.0,84.0,23.10,85.0,85.0,0.0


In [ ]:
import os
import random
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_recall_curve, auc

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek

import matplotlib.pyplot as plt


In [ ]:

df = pd.read_csv(os.path.join(root_path, "framingham.csv"))


print("Ukuran dataset:", df.shape)
print("\nJumlah missing value per kolom:\n", df.isnull().sum())


print("\nInfo dataset:")
print(df.info())

if "TenYearCHD" in df.columns:
    print("\nDistribusi label target (TenYearCHD):")
    print(df["TenYearCHD"].value_counts(normalize=True))


Ukuran dataset: (4240, 16)

Jumlah missing value per kolom:
 male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

Info dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4240 non-null   int64  
 1   age              4240 non-null   int64  
 2   education        4135 non-null   float64
 3   currentSmoker    4240 non-null   int64  
 4   cigsPerDay       4211 non-null   float64
 5   BPMeds           4187 non-null   float64
 6   prevalentStroke  4240 non-null   int64  
 7   prevalentHyp     

In [ ]:

from sklearn.impute import SimpleImputer

num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(exclude=[np.number]).columns

imputer_num = SimpleImputer(strategy="median")
df[num_cols] = imputer_num.fit_transform(df[num_cols])

if len(cat_cols) > 0:
    imputer_cat = SimpleImputer(strategy="most_frequent")
    df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

print("Jumlah missing value setelah imputasi:\n", df.isnull().sum().sum())


Jumlah missing value setelah imputasi:
 0


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop("TenYearCHD", axis=1)
y = df["TenYearCHD"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=True, stratify=y, random_state=42
)

print("Distribusi kelas di train:", np.bincount(y_train))
print("Distribusi kelas di test :", np.bincount(y_test))


Distribusi kelas di train: [2517  451]
Distribusi kelas di test : [1079  193]


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
print("Jumlah sampel sebelum SMOTE-IPF:", X_train.shape)
print("Distribusi kelas sebelum SMOTE-IPF:", np.bincount(y_train))

Jumlah sampel sebelum SMOTE-IPF: (2968, 15)
Distribusi kelas sebelum SMOTE-IPF: [2517  451]


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Baseline SVM (tanpa SMOTE/IPF)
svm = SVC(kernel='rbf', C=10, gamma=0.1, class_weight='balanced', random_state=42)  # random_state biar konsisten
svm.fit(X_train_scaled, y_train)
y_pred = svm.predict(X_test_scaled)

print("=== Baseline SVM (Tanpa SMOTE-IPF) ===")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


=== Baseline SVM (Tanpa SMOTE-IPF) ===
Classification Report:
               precision    recall  f1-score   support

         0.0       0.88      0.78      0.83      1079
         1.0       0.24      0.38      0.29       193

    accuracy                           0.72      1272
   macro avg       0.56      0.58      0.56      1272
weighted avg       0.78      0.72      0.74      1272

F1-Score: 0.2908366533864542
Confusion Matrix:
 [[843 236]
 [120  73]]


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from imblearn.pipeline import Pipeline

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
pipeline_svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel='rbf', C=10, gamma=0.1, class_weight='balanced', random_state=42))
])

scores = cross_val_score(pipeline_svm, X, y, cv=cv, scoring="f1")
print("Rata-rata F1-Score CV (Tanpa SMOTE-IPF):", scores.mean())


Rata-rata F1-Score CV (Tanpa SMOTE-IPF): 0.296993521523254


In [ ]:


# ---------- Quick tuned SVM on SMOTE-resampled data (optional quick check) ----------
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

svm_smote = SVC(kernel='rbf', C=10, gamma=0.1, class_weight='balanced', random_state=42)
svm_smote.fit(X_resampled, y_resampled)
y_pred_smote = svm_smote.predict(X_test_scaled)

print("\n=== SVM + SMOTE (Tuned quick check) ===")
print(classification_report(y_test, y_pred_smote))
print("F1-Score:", f1_score(y_test, y_pred_smote))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_smote))


# ---------- GridSearchCV on pipeline (recommended) ----------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

pipeline_smote_svm = ImbPipeline([
    ("scaler", StandardScaler()),                # scaling BEFORE SMOTE is recommended
    ("smote", SMOTE(random_state=42)),
    ("svm", SVC(probability=True, random_state=42))  # probability=True for threshold tuning / PR curve
])

# If you want a faster run (short) use the smaller_grid; for final run use full_grid
smaller_grid = {
    "smote__k_neighbors": [3, 5],
    "smote__sampling_strategy": [0.75, 1.0],
    "svm__C": [1, 10],
    "svm__gamma": ["scale", 0.1],
    "svm__kernel": ["rbf"]
}

full_grid = {
    "smote__k_neighbors": [3, 5, 7],
    "smote__sampling_strategy": [0.5, 0.75, 1.0],
    "svm__C": [0.1, 1, 10],
    "svm__gamma": ["scale", 0.1, 1],
    "svm__kernel": ["rbf", "linear"],
    # you can also toggle class_weight if desired:
    # "svm__class_weight": [None, "balanced"]
}

grid = GridSearchCV(
    estimator=pipeline_smote_svm,
    param_grid=smaller_grid,    # ganti ke full_grid kalau mau run lengkap
    scoring="f1",
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Jalankan grid search pada X_train (JANGAN pake X_train_scaled karena pipeline melakukan scaling)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV F1:", grid.best_score_)

# Evaluasi di test set (gunakan pipeline.best_estimator_ — pipeline sudah berisi scaler & SMOTE)
best_pipe = grid.best_estimator_
y_test_proba = best_pipe.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_proba >= 0.5).astype(int)   # default threshold

print("=== SVM + SMOTE (Grid best) - Test set (th=0.5) ===")
print(classification_report(y_test, y_test_pred))
print("F1 (test):", f1_score(y_test, y_test_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_test_pred))




=== SVM + SMOTE (Tuned quick check) ===
              precision    recall  f1-score   support

         0.0       0.87      0.80      0.83      1079
         1.0       0.22      0.33      0.26       193

    accuracy                           0.72      1272
   macro avg       0.55      0.56      0.55      1272
weighted avg       0.77      0.72      0.74      1272

F1-Score: 0.2647058823529412
Confusion Matrix:
 [[859 220]
 [130  63]]
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best params: {'smote__k_neighbors': 5, 'smote__sampling_strategy': 0.75, 'svm__C': 1, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Best CV F1: 0.33058612543075977
=== SVM + SMOTE (Grid best) - Test set (th=0.5) ===
              precision    recall  f1-score   support

         0.0       0.89      0.79      0.84      1079
         1.0       0.27      0.42      0.33       193

    accuracy                           0.74      1272
   macro avg       0.58      0.61      0.58      1272
weighted avg 

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
import numpy as np

class SMOTE_IPF(BaseEstimator):

    def __init__(self, smote_k=5, ipf_k=3, max_iter=5, sampling_strategy=1.0,
                 random_state=None, verbose=False, remove_only_synthetic=True):
        self.smote_k = smote_k
        self.ipf_k = ipf_k
        self.max_iter = max_iter
        self.sampling_strategy = sampling_strategy
        self.random_state = random_state
        self.verbose = verbose
        self.remove_only_synthetic = remove_only_synthetic

    def fit_resample(self, X, y):
        # ensure numpy arrays
        X = np.asarray(X)
        y = np.asarray(y).astype(int).ravel()

        n_orig = X.shape[0]

        # create SMOTE instance here so it uses current params (GridSearch can change them)
        smote = SMOTE(k_neighbors=self.smote_k,
                      sampling_strategy=self.sampling_strategy,
                      random_state=self.random_state)

        X_res, y_res = smote.fit_resample(X, y)

        # Track which rows are original vs synthetic
        # Strategy: first n_orig rows correspond to original samples,
        # SMOTE appends synthetic samples after original ones.
        n_after_smote = X_res.shape[0]
        n_synthetic = n_after_smote - n_orig
        synthetic_mask = np.zeros(n_after_smote, dtype=bool)
        if n_synthetic > 0:
            synthetic_mask[n_orig:] = True  # mark synthetic samples

        # combine X and y
        data = np.hstack((X_res.copy(), y_res.reshape(-1, 1)))

        if self.verbose:
            unique, counts = np.unique(y_res, return_counts=True)
            print(f"[SMOTE] before IPF: total={data.shape[0]}, class_dist={dict(zip(unique, counts))}, synthetic={n_synthetic}")

        # Iterative Partitioning Filter (IPF)
        for i in range(self.max_iter):
            clf = KNeighborsClassifier(n_neighbors=max(1, self.ipf_k))
            X_curr = data[:, :-1]
            y_curr = data[:, -1]

            clf.fit(X_curr, y_curr)
            y_pred = clf.predict(X_curr)

            misclassified = (y_pred != y_curr)

            if not np.any(misclassified):
                if self.verbose:
                    print(f"[IPF] iter {i+1}: no misclassified samples -> stop")
                break

            if self.remove_only_synthetic:
                # build synthetic_mask for current data size (some synthetic may have been removed)
                # we compute positions of original rows: first n_orig remain original until removed
                current_len = data.shape[0]
                # recalc which indices are synthetic: if current_len <= n_orig then none synthetic
                current_synthetic_mask = np.zeros(current_len, dtype=bool)
                if current_len > n_orig:
                    current_synthetic_mask[n_orig:current_len] = True

                # only remove misclassified & synthetic
                removable = misclassified & current_synthetic_mask
                removed_count = removable.sum()

                # if none removable (only original misclassified) -> stop or remove anyway?
                if removed_count == 0:
                    if self.verbose:
                        print(f"[IPF] iter {i+1}: misclassified mostly original samples -> stop to avoid removing originals")
                    break

                data = data[~removable]
                # adjust n_orig if originals were removed (we avoid removing originals here so n_orig stays)
            else:
                # remove all misclassified rows (both synthetic and originals)
                removed_count = misclassified.sum()
                data = data[~misclassified]

            if self.verbose:
                unique, counts = np.unique(data[:, -1], return_counts=True)
                print(f"[IPF] iter {i+1}: removed={removed_count}, remain_total={data.shape[0]}, class_dist={dict(zip(unique, counts))}")

        X_final = data[:, :-1]
        y_final = data[:, -1].astype(int)

        if self.verbose:
            unique, counts = np.unique(y_final, return_counts=True)
            print(f"[SMOTE-IPF] final: total={X_final.shape[0]}, class_dist={dict(zip(unique, counts))}")

        return X_final, y_final


In [ ]:
# Tambahkan cetak rate penghapusan
smote_ipf = SMOTE_IPF(smote_k=5, ipf_k=3, max_iter=5, random_state=42, verbose=True)
X_resampled_ipf, y_resampled_ipf = smote_ipf.fit_resample(X_train_scaled, y_train)


print("Jumlah setelah SMOTE-IPF:", X_resampled_ipf.shape)
print("Distribusi kelas setelah SMOTE-IPF:", np.bincount(y_resampled_ipf))



[SMOTE] before IPF: total=5034, class_dist={np.int64(0): np.int64(2517), np.int64(1): np.int64(2517)}, synthetic=2066
[IPF] iter 1: removed=5, remain_total=5029, class_dist={np.float64(0.0): np.int64(2517), np.float64(1.0): np.int64(2512)}
[IPF] iter 2: misclassified mostly original samples -> stop to avoid removing originals
[SMOTE-IPF] final: total=5029, class_dist={np.int64(0): np.int64(2517), np.int64(1): np.int64(2512)}
Jumlah setelah SMOTE-IPF: (5029, 15)
Distribusi kelas setelah SMOTE-IPF: [2517 2512]


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_recall_curve, auc
import numpy as np

# 1) Train SVM pada data hasil SMOTE-IPF (sudah discale)
svm_resampled = SVC(kernel='rbf', C=10, gamma=0.1, class_weight='balanced', probability=True, random_state=42)
svm_resampled.fit(X_resampled_ipf, y_resampled_ipf)

# 2) Prediksi di test set (gunakan X_test_scaled)
y_pred_resampled = svm_resampled.predict(X_test_scaled)
y_proba_resampled = svm_resampled.predict_proba(X_test_scaled)[:, 1]

# 3) Evaluasi lengkap
print("=== SVM + SMOTE-IPF (Test set) ===")
print("Classification Report:\n", classification_report(y_test, y_pred_resampled))
print("F1-Score (pos=1):", f1_score(y_test, y_pred_resampled, pos_label=1))
print("F1-Score (macro):", f1_score(y_test, y_pred_resampled, average='macro'))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_resampled))

# 4) Precision-Recall AUC (berguna untuk dataset imbalanced)
prec, rec, thr = precision_recall_curve(y_test, y_proba_resampled)
pr_auc = auc(rec, prec)
print("PR-AUC:", pr_auc)



=== SVM + SMOTE-IPF (Test set) ===
Classification Report:
               precision    recall  f1-score   support

         0.0       0.87      0.80      0.83      1079
         1.0       0.22      0.33      0.26       193

    accuracy                           0.72      1272
   macro avg       0.55      0.56      0.55      1272
weighted avg       0.77      0.72      0.74      1272

F1-Score (pos=1): 0.2641509433962264
F1-Score (macro): 0.5471698113207547
Confusion Matrix:
 [[858 221]
 [130  63]]
PR-AUC: 0.2100793708745129
